In [ ]:
!pip install datasets
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
import torch
import random
import tensorflow as tf
from datasets import Dataset
import tqdm as notebook_tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score

df_test = pd.read_csv("politicES_phase_2_test_codalab.csv", on_bad_lines='skip')
df_test['tweet'] = df_test.apply(lambda row: row['gender'] + " " + row['profession']+ " " + row['tweet'], axis=1)
df_test.drop(['ideology_multiclass','gender','profession','label'], axis=1, inplace=True)


In [ ]:
df_test

,ideology_multiclass,tweet
0,moderate_right,"male journalist right Ayer, en Guadalajara: fe..."
1,moderate_right,"male journalist right “Chile Vamos”, o sea, la..."
2,moderate_right,male journalist right Se está tratando de sile...
3,moderate_right,"male journalist right @user Lucía, lo siento p..."
4,moderate_right,male journalist right JxC no dio quórum para e...
...,...,...
43755,right,"male politician right No, si ahora resulta que..."
43756,right,male politician right Tenemos un buen futuro s...
43757,right,male politician right Cuando el [POLITICAL_PAR...
43758,right,male politician right La entrevista de @user a...


In [ ]:
# Initialize the tokenizer
checkpoint_name= 'juan-glez29/BERTuit-ideologiabin-two'
tokenizer = AutoTokenizer.from_pretrained(checkpoint_name)
# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/483k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/283k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_test['ideology_binary'] = label_encoder.fit_transform(df_test['ideology_binaryender'])

In [ ]:
df_test['tweet'] = df_test['tweet'].apply(lambda x: x.replace("@user", "<usr>"))

In [ ]:
def tokenize_function(example):
    return tokenizer(example['tweet'])

# Conversión del DataFrame de pandas a un Dataset de Hugging Face para pasarlo al modelo
huggingface_dataset = Dataset.from_pandas(df_test)
huggingface_dataset

Dataset({
    features: ['ideology_multiclass', 'tweet'],
    num_rows: 43760
})

In [ ]:
# Aplicamos la tokenización a todo el dataset
tokenized_dataset = huggingface_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns("tweet")
test_dataset = tokenized_dataset.rename_column("ideology_binary", "labels")
test_dataset

Map:   0%|          | 0/43760 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 43760
})

In [ ]:
# Define test trainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
test_trainer = Trainer(model, data_collator=data_collator, tokenizer=tokenizer)

predicted_results = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = predicted_results.predictions.argmax(-1) # Get the highest probability prediction

labels = test_dataset["labels"]

macro_f1 = f1_score(labels, y_pred, average='macro')# o weighted si estan desbalceadas las clases

print(f"F1-Score:\t{macro_f1}")

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


F1-Score:	0.6266239437189619


In [ ]:
# # Tokenize text inputs
# tokenized_inputs = tokenizer(df_test['tweet'][6], return_tensors="pt")

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)
# tokenized_inputs = tokenized_inputs.to(device)

# # Pass tokenized inputs through the Transformer model
# outputs = model(**tokenized_inputs)

# # Get predictions
# labels = ['izq', 'izq_mod', 'dcha_mod', 'dcha']
# prediction = torch.argmax(outputs.logits)

# print(outputs)

# print(f"The prediction is {labels[prediction]}")

SequenceClassifierOutput(loss=None, logits=tensor([[-3.7482, -4.5807,  2.8838,  4.7698]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
The prediction is dcha
